In [1]:
import pypsa
import pandas as pd
import xarray as xr
n = pypsa.Network("elec_s_37_ec_lcopt_Co2L0.0-1H.nc")

print(n)

INFO:pypsa.io:Imported network elec_s_37_ec_lcopt_Co2L0.0-24H.nc has buses, carriers, generators, global_constraints, lines, links, loads, shapes, storage_units, stores


PyPSA Network
Components:
 - Bus: 111
 - Carrier: 25
 - Generator: 265
 - GlobalConstraint: 1
 - Line: 52
 - Link: 198
 - Load: 37
 - Shape: 123
 - StorageUnit: 50
 - Store: 74
Snapshots: 366


In [2]:
import pandas as pd

generation_data = n.generators_t.p
filtered_generation_data = generation_data.loc[~generation_data.index.isin(['2020-02-29'])]

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += filtered_generation_data[gen_id].sum()

print(carrier_generation)

offwind-ac       2.767332e+06
offwind-float    7.366310e+05
oil              0.000000e+00
onwind           2.797454e+07
ror              6.967353e+06
solar            4.666162e+07
solar-hsat       5.253987e+06
CCGT             0.000000e+00
lignite          0.000000e+00
offwind-dc       5.021878e+06
coal             0.000000e+00
nuclear          4.786971e+06
biomass          9.780354e+05
OCGT             0.000000e+00
dtype: float64


/var/folders/gy/z7x44kpn24d18spbg3w6_5fr0000gn/T/ipykernel_70341/967269348.py:5: FutureWarning:

The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.



In [3]:
import pandas as pd

storage_data = n.storage_units_t.p

filtered_storage_data = storage_data.loc[~storage_data.index.isin(['2020-02-29'])]

carrier_storage = pd.Series(0.0, index=n.storage_units.carrier.unique())

for unit_id, storage_unit in n.storage_units.iterrows():
    carrier = storage_unit['carrier']
    carrier_storage[carrier] += filtered_storage_data[unit_id].sum()

print(carrier_storage)

hydro    1.548989e+07
PHS      1.214294e+04
dtype: float64


/var/folders/gy/z7x44kpn24d18spbg3w6_5fr0000gn/T/ipykernel_70341/3887856627.py:6: FutureWarning:

The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.



In [4]:
n.add("Carrier", "SBSP", co2_emissions=0.0)

ac_buses = n.buses[n.buses['carrier'] == 'AC']

for bus in ac_buses.index:
    n.add("Generator", f"SBSP Generator at {bus}", 
          bus=bus,  
          carrier="SBSP",  
          p_nom_min=0.0,  # MW
          marginal_cost=0.0,  # EUR/MWh
          capital_cost=267869.9558,  # EUR/MW
          p_nom_extendable=True  
         )

In [5]:
n.generators['p_nom_extendable'] = True

target_generators = n.generators[
    n.generators['type'].isin(['geothermal', 'biomass', 'ror', 'nuclear'])
]

n.generators.loc[target_generators.index, 'p_nom_min'] = target_generators['p_nom']

carriers_to_modify = ["oil", "lignite", "coal", "CCGT", "OCGT"]

n.generators.loc[n.generators['carrier'].isin(carriers_to_modify), 'p_nom_min'] = 0

n.storage_units['p_nom_extendable'] = True

n.storage_units['p_nom_max'] = n.storage_units['p_nom']

In [6]:
snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|████████████| 15/15 [00:00<00:00, 32.55it/s]
INFO:linopy.io: Writing time: 3.38s


Read LP format model from file /private/var/folders/gy/z7x44kpn24d18spbg3w6_5fr0000gn/T/linopy-problem-n_i2i55s.lp


INFO:gurobipy:Read LP format model from file /private/var/folders/gy/z7x44kpn24d18spbg3w6_5fr0000gn/T/linopy-problem-n_i2i55s.lp


Reading time = 1.27 seconds


INFO:gurobipy:Reading time = 1.27 seconds


...

Solved in 313014 iterations and 420.23 seconds (297.01 work units)


INFO:gurobipy:Solved in 313014 iterations and 420.23 seconds (297.01 work units)


Optimal objective  2.826184432e+10


INFO:gurobipy:Optimal objective  2.826184432e+10
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 321293 primals, 663113 duals
Objective: 2.83e+10
Solver model: available
Solver message: 2

/opt/anaconda3/lib/python3.11/site-packages/pypsa/optimization/optimize.py:357: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-l

('ok', 'optimal')

In [13]:
pd.set_option('display.max_columns', None)

print(n.storage_units)

            carrier    bus         p_nom   capital_cost      max_hours  \
StorageUnit                                                              
AL1 0 hydro   hydro  AL1 0   1690.700000  216671.771451      42.573609   
AT1 0 PHS       PHS  AT1 0   5057.300000  216671.771451     309.164904   
AT1 0 hydro   hydro  AT1 0   3457.620000  216671.771451     925.492102   
BA1 0 PHS       PHS  BA1 0    440.000000  216671.771451       0.007907   
BA1 0 hydro   hydro  BA1 0   1614.500000  216671.771451    1548.467018   
BE1 0 PHS       PHS  BE1 0   1308.000000  216671.771451       4.365443   
BE1 0 hydro   hydro  BE1 0      1.900000  216671.771451  105263.157895   
BG1 0 PHS       PHS  BG1 0    864.000000  216671.771451      31.643519   
BG1 0 hydro   hydro  BG1 0   1925.140000  216671.771451    2077.770967   
CH1 0 PHS       PHS  CH1 0   5081.350000  216671.771451      68.632264   
CH1 0 hydro   hydro  CH1 0   9786.310000  216671.771451     894.597281   
DE1 0 PHS       PHS  DE1 0   7134.2200

In [5]:
n.statistics().dropna()

Optimal Capacity  Installed Capacity  \
Generator   Combined-Cycle Gas           169023.489186       169023.489186   
            Offshore Wind (AC)             4751.233333         4751.233333   
            Offshore Wind (DC)             3627.533333         3627.533333   
            Offshore Wind (Floating)       4751.233333         4751.233333   
            Onshore Wind                 115853.150278        89455.181147   
            Open-Cycle Gas                   47.150000           47.150000   
            Run of River                  51522.370043        51522.370043   
            SBSP                          62682.132350            0.000000   
            Solar                        214237.851514        67523.708588   
            biomass                       17029.158976        17029.158976   
            coal                          73416.104700        73416.104700   
            geothermal                       98.000000           98.000000   
            lignite                       39939.732918        39939.732918   
            nuclear                      119412.100000       119412.100000   
            oil                            4686.760390         4686.760390   
            solar-hsat                   141869.098098            0.000000   
Line        AC                           276119.666826       273550.924678   
Link        DC                            30306.984334        20491.000000   
StorageUnit Pumped Hydro Storage          54837.473000        54837.473000   
            Reservoir & Dam               99387.650991        99387.650991   

                                      Capacity Factor      Dispatch  \
Generator   Combined-Cycle Gas               0.000000  0.000000e+00   
            Offshore Wind (AC)               0.533052  2.218606e+07   
            Offshore Wind (DC)               0.562494  1.787449e+07   
            Offshore Wind (Floating)         0.427401  1.778877e+07   
            Onshore Wind                     0.324255  3.290777e+08   
            Open-Cycle Gas                   0.000000  0.000000e+00   
            Run of River                     0.372358  1.680584e+08   
            SBSP                             0.999989  5.490893e+08   
            Solar                            0.153561  2.881917e+08   
            biomass                          0.343319  5.121477e+07   
            coal                             0.000000  0.000000e+00   
            geothermal                       0.000000  0.000000e+00   
            lignite                          0.000000  0.000000e+00   
            nuclear                          0.673691  7.047148e+08   
            oil                              0.000000  0.000000e+00   
            solar-hsat                       0.208492  2.591078e+08   
Line        AC                               0.327337  0.000000e+00   
Link        DC                               0.841329  0.000000e+00   
StorageUnit Pumped Hydro Storage             0.098764 -6.777697e+06   
            Reservoir & Dam                  0.388268  3.380402e+08   

                                      Transmission    Withdrawal  \
Generator   Combined-Cycle Gas        0.000000e+00  0.000000e+00   
            Offshore Wind (AC)        0.000000e+00  0.000000e+00   
            Offshore Wind (DC)        0.000000e+00  0.000000e+00   
            Offshore Wind (Floating)  0.000000e+00  0.000000e+00   
            Onshore Wind              0.000000e+00  0.000000e+00   
            Open-Cycle Gas            0.000000e+00  0.000000e+00   
            Run of River              0.000000e+00  0.000000e+00   
            SBSP                      0.000000e+00  0.000000e+00   
            Solar                     0.000000e+00  0.000000e+00   
            biomass                   0.000000e+00  0.000000e+00   
            coal                      0.000000e+00  0.000000e+00   
            geothermal                0.000000e+00  0.000000e+00   
            lignite    

In [7]:
import pandas as pd
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)

offwind-ac       8.409366e+05
offwind-float    6.813108e+05
oil              0.000000e+00
onwind           1.168251e+07
ror              0.000000e+00
solar            9.225396e+06
solar-hsat       0.000000e+00
CCGT             0.000000e+00
lignite          0.000000e+00
offwind-dc       5.958419e+05
coal             0.000000e+00
nuclear          1.366385e+06
biomass          3.361873e+05
OCGT             0.000000e+00
SBSP             7.434222e+07
dtype: float64


In [8]:
SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)

Buses with active SBSP generators:
['AL1 0' 'AT1 0' 'BA1 0' 'BE1 0' 'BG1 0' 'CH1 0' 'CZ1 0' 'DE1 0' 'ES1 0'
 'FI2 0' 'FR1 0' 'GB0 0' 'GR1 0' 'HU1 0' 'IE5 0' 'IT1 0' 'LT6 0' 'LU1 0'
 'NL1 0' 'NO2 0' 'PL1 0' 'PT1 0' 'RO1 0' 'RS1 0' 'SE2 0' 'SI1 0' 'SK1 0']


In [9]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)

Total p_nom_opt for all SBSP generators: 203140.05641142995


In [10]:
import pandas as pd

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)

Generator
SBSP Generator at AL1 0      429.439835
SBSP Generator at AT1 0     6393.715541
SBSP Generator at BA1 0     2470.738360
SBSP Generator at BE1 0    10855.296878
SBSP Generator at BG1 0     1529.089408
SBSP Generator at CH1 0     2013.680498
SBSP Generator at CZ1 0     5101.435540
SBSP Generator at DE1 0    22850.849457
SBSP Generator at DK1 0        0.000000
SBSP Generator at DK2 0        0.000000
SBSP Generator at EE6 0        0.000000
SBSP Generator at ES1 0    14502.860431
SBSP Generator at ES4 0        0.000000
SBSP Generator at FI2 0     9024.097340
SBSP Generator at FR1 0    36335.082944
SBSP Generator at GB0 0    15396.336771
SBSP Generator at GB5 0        0.000000
SBSP Generator at GR1 0      142.429618
SBSP Generator at HR1 0        0.000000
SBSP Generator at HU1 0     3711.237531
SBSP Generator at IE5 0     3392.939685
SBSP Generator at IT1 0    19032.354555
SBSP Generator at IT3 0        0.000000
SBSP Generator at LT6 0     1812.238350
SBSP Generator at LU1 0     11

In [11]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 100000  

snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [14]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)

offwind-ac       3.245344e+05
offwind-float    2.266948e+05
oil              0.000000e+00
onwind           3.156805e+06
ror              0.000000e+00
solar            7.896561e+05
solar-hsat       0.000000e+00
CCGT             0.000000e+00
lignite          0.000000e+00
offwind-dc       1.285147e+05
coal             0.000000e+00
nuclear          9.186225e+05
biomass          1.547583e+05
OCGT             0.000000e+00
SBSP             1.087807e+08
dtype: float64
Buses with active SBSP generators:
['AL1 0' 'AT1 0' 'BA1 0' 'BE1 0' 'BG1 0' 'CH1 0' 'CZ1 0' 'DE1 0' 'DK2 0'
 'ES1 0' 'ES4 0' 'FI2 0' 'FR1 0' 'GB0 0' 'GB5 0' 'GR1 0' 'HR1 0' 'HU1 0'
 'IE5 0' 'IT1 0' 'IT3 0' 'LT6 0' 'LU1 0' 'ME1 0' 'MK1 0' 'NL1 0' 'NO2 0'
 'PL1 0' 'PT1 0' 'RO1 0' 'RS1 0' 'SE2 0' 'SI1 0' 'SK1 0']
Total p_nom_opt for all SBSP generators: 321297.55838895007
Generator
SBSP Generator at AL1 0     2249.516644
SBSP Generator at AT1 0    10554.429712
SBSP Generator at BA1 0     2110.307345
SBSP Generator at BE1 0    17155.

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 150000 
snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [14]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)


import pandas as pd

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']
print(sbsp_p_nom_opt)

offwind-ac       3.245344e+05
offwind-float    2.266948e+05
oil              0.000000e+00
onwind           3.156805e+06
ror              0.000000e+00
solar            7.896561e+05
solar-hsat       0.000000e+00
CCGT             0.000000e+00
lignite          0.000000e+00
offwind-dc       1.285147e+05
coal             0.000000e+00
nuclear          9.186225e+05
biomass          1.547583e+05
OCGT             0.000000e+00
SBSP             1.087807e+08
dtype: float64
Buses with active SBSP generators:
['AL1 0' 'AT1 0' 'BA1 0' 'BE1 0' 'BG1 0' 'CH1 0' 'CZ1 0' 'DE1 0' 'DK2 0'
 'ES1 0' 'ES4 0' 'FI2 0' 'FR1 0' 'GB0 0' 'GB5 0' 'GR1 0' 'HR1 0' 'HU1 0'
 'IE5 0' 'IT1 0' 'IT3 0' 'LT6 0' 'LU1 0' 'ME1 0' 'MK1 0' 'NL1 0' 'NO2 0'
 'PL1 0' 'PT1 0' 'RO1 0' 'RS1 0' 'SE2 0' 'SI1 0' 'SK1 0']
Total p_nom_opt for all SBSP generators: 321297.55838895007
Generator
SBSP Generator at AL1 0     2249.516644
SBSP Generator at AT1 0    10554.429712
SBSP Generator at BA1 0     2110.307345
SBSP Generator at BE1 0    17155.

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 200000 

snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 250000 

snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)



sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']


sbsp_p_nom_opt = sbsp_generators['p_nom_opt']


print(sbsp_p_nom_opt)

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 300000  

snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']


total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)


sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']


n.generators.loc[sbsp_generators.index, 'capital_cost'] = 350000  # 
snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)

import pandas as pd

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)

In [ ]:
sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

n.generators.loc[sbsp_generators.index, 'capital_cost'] = 400000  

snapshots_to_optimize = n.snapshots[:8784]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
generation_data = n.generators_t.p

carrier_generation = pd.Series(0.0, index=n.generators.carrier.unique())

for gen_id, generator in n.generators.iterrows():
    carrier = generator['carrier']
    carrier_generation[carrier] += generation_data[gen_id].sum()

print(carrier_generation)



SBSP_generators = n.generators[n.generators['carrier'] == 'SBSP']

SBSP_generation = n.generators_t.p[SBSP_generators.index]

active_SBSP_generators = SBSP_generation.loc[:, (SBSP_generation != 0).any(axis=0)]

active_SBSP_buses = SBSP_generators.loc[active_SBSP_generators.columns, 'bus'].unique()

print("Buses with active SBSP generators:")
print(active_SBSP_buses)

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

total_p_nom_opt_sbsp = sbsp_generators['p_nom_opt'].sum()

print("Total p_nom_opt for all SBSP generators:", total_p_nom_opt_sbsp)

import pandas as pd

sbsp_generators = n.generators[n.generators['carrier'] == 'SBSP']

sbsp_p_nom_opt = sbsp_generators['p_nom_opt']

print(sbsp_p_nom_opt)